In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [17]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [18]:
# encoders and decoders = tokenizer
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [19]:
# training and validation splits
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data= train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print(f'\nsize x matrix {x.shape}\n')
print('inputs: ')
print(x)
print('targets: ')
print(y)


size x matrix torch.Size([4, 8])

inputs: 
tensor([[67, 57, 65, 58,  1, 73, 68,  1],
        [71, 58, 72,  0, 72, 61, 74, 57],
        [ 1, 69, 58, 68, 69, 65, 58,  1],
        [71, 58,  1, 74, 72,  1, 66, 74]])
targets: 
tensor([[57, 65, 58,  1, 73, 68,  1, 73],
        [58, 72,  0, 72, 61, 74, 57, 57],
        [69, 58, 68, 69, 65, 58,  1, 60],
        [58,  1, 74, 72,  1, 66, 74, 56]])


In [20]:
# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print('when input is', context, 'target is ', target)

# how to measure the loss:
# -ln(1/80) = to predict the next carachter. // have to minimize the loss, the objective
# utilize adamW as the optimize algorithm

In [21]:
@torch.no_grad() # certificates that pytorch doesn't use any gradient here
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [22]:
class BigramLanguageModel(nn.Module):

    def __init__ (self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits =  self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the laste time step
            logits = logits[:, -1, :] # becomes(B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



m7f7KvK6)S[A"k-.:MRl?CXnL&_I5M44oIgV;zf*
O5zy[1tj_oT:r:BC9[(mccnP2-POcf*RenPxKr1kYFR'v-A-0dS;jLVPk.PdmYQwG1
SZWG1R2M,MmHPInR_9Z6IsrR;C
XrdGXnyVPkNnnJV(Stw olW64;KaCi7GkW)de!ahd'1y'UmzZD8V2(M2UbEizT1rSJgG1lqM.W[j-_ACha XYQ*0TnL
bb9FhDyLZAF
5';C"gLEM8m_p,2lC9:Tfx;bdH
kqrg:'ofD&2cE.9Me80EI]8:HkYIugk
KW((_A'2M4*e*Axsr7D v2M9')OlUi2&;zUeHPJn,wkP4YQBZgq.Hl'V]6ZqW.__PU0_Q6IR"uamwKaNG1jx9&YIv)cq?:&4Lhg_b'-x;QM7EY!Cq,g,G;pjE*
5qf?gW9 mFUVBHC ;"LeA
jSBR,8lx",o,0hv]d*cx1W_Hoon5ZBBtBENT8T0,,W6OfB3C,Q"
dx)wl


In [36]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range (max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, val loss: {losses["val"]:.4f}')


    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logit, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step: 0, train loss: 2.4449, val loss: 2.4685
step: 250, train loss: 2.4384, val loss: 2.4690
step: 500, train loss: 2.4390, val loss: 2.4976
step: 750, train loss: 2.4726, val loss: 2.4987
step: 1000, train loss: 2.4597, val loss: 2.4728
step: 1250, train loss: 2.4228, val loss: 2.4785
step: 1500, train loss: 2.4324, val loss: 2.4913
step: 1750, train loss: 2.4253, val loss: 2.4820
step: 2000, train loss: 2.4110, val loss: 2.4705
step: 2250, train loss: 2.4454, val loss: 2.4662
step: 2500, train loss: 2.4540, val loss: 2.4980
step: 2750, train loss: 2.4360, val loss: 2.4994
step: 3000, train loss: 2.4341, val loss: 2.4910
step: 3250, train loss: 2.4471, val loss: 2.4778
step: 3500, train loss: 2.4481, val loss: 2.4786
step: 3750, train loss: 2.4508, val loss: 2.4746
step: 4000, train loss: 2.4355, val loss: 2.4454
step: 4250, train loss: 2.4516, val loss: 2.5039
step: 4500, train loss: 2.4467, val loss: 2.4746
step: 4750, train loss: 2.4240, val loss: 2.5181
step: 5000, train loss: 2.

In [37]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ly Bathed," mme burfiny I't Jorinsporey coorefur arotonthe ly
"alith o entyoghangaberighed E n p Uhe gy bare b, nyod whadllomatod trmy Soris vedr
" salld w ththe wn is d asurot f ouisthithe T lins k fftelit.
thoor ghafare y ge  cothondovened, ma U; tas f stick towheirore ask s higrkewa berrotld illlichie
Ze Tn l islebe m bomarosche, twedvasouree t wis cee tll t


tif b thint ulil boutt I'toty, aidd llyltl l dly, or as,
whtl, thev]mary "If ickiergly'smarigey," tathiexcackad s  ak it ys.

arofuin 
